In [1]:
# as png
# %matplotlib inline 
#  interactable inside ide
# %matplotlib widget
### interactable seperate window
%matplotlib tk 

In [14]:
import os
import sys
sys.path.append('/media/mangaldeep/HDD2/workspace/MotionControl_MasterThesis')
os.getcwd()

'/media/mangaldeep/HDD2/workspace/MotionControl_MasterThesis/notebooks'

In [15]:
import mne
import numpy as np
import pandas as pd
import plotly.express as px
import chart_studio.plotly as py
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import libs
import data.brain_atlas as bm

In [ ]:
# physionet = 'https://physionet.org/files/eegmmidb/1.0.0/'
    ''' 
    64 - Channel EEG
    109 - Volunteers
    14 - Experiments (Each recording 1 to 2 minutes) 
            T1: Left/Fist,    T2: Right/Feet
            1.Baseline, eyes open (T0)
            2.Baseline, eyes closed (T0)
            3.Task 1 (open and close left or right fist) (T1/T2)
            4.Task 2 (imagine opening and closing left or right fist) (T1/T2)
            5.Task 3 (open and close both fists or both feet) (T1/T2)
            6.Task 4 (imagine opening and closing both fists or both feet) (T1/T2)
            7.Task 1 (T1/T2)
            8.Task 2 (T1/T2)
            9.Task 3 (T1/T2)
            10.Task 4 (T1/T2)
            11.Task 1 (T1/T2)
            12.Task 2 (T1/T2)
            13.Task 3 (T1/T2)
            14.Task 4 (T1/T2)
    Annotataions:
        T0 corresponds to rest
        T1 corresponds to onset of motion (real or imagined) of
            the left fist (in runs 3, 4, 7, 8, 11, and 12)
            both fists (in runs 5, 6, 9, 10, 13, and 14)
        T2 corresponds to onset of motion (real or imagined) of
            the right fist (in runs 3, 4, 7, 8, 11, and 12)
            both feet (in runs 5, 6, 9, 10, 13, and 14)
    '''

In [ ]:
# Mapping to the task in  hand
'''
T1 - Left = Take Left
T2 - Right = Take Right
T1 - Fist = Brake
T2 - Feet = Accelerate
'''

In [4]:
runs = list(range(1,15))
person_id = 5
# path2 = '/media/mangaldeep/HDD3/DataSets/Physionet'
path = '/media/mangaldeep/HDD3/DataSets/mne_data'
fname = mne.datasets.eegbci.load_data(person_id, runs, path=path)

In [5]:
    # T0 corresponds to rest
    # T1 corresponds to onset of motion (real or imagined) of
    #     the left fist (in runs 3, 4, 7, 8, 11, and 12)
    #     both fists (in runs 5, 6, 9, 10, 13, and 14)
    # T2 corresponds to onset of motion (real or imagined) of
    #     the right fist (in runs 3, 4, 7, 8, 11, and 12)
    #     both feet (in runs 5, 6, 9, 10, 13, and 14)
run_id = 3
raw = mne.io.read_raw_edf(fname[run_id], preload=True, verbose=False)

#### Channel Name Modification

In [6]:
# Dataset lacks EEG electrode locations. 
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')

# Montage channel names are different from dataset
# FP3 --> Fp3
ch_loc = ten_twenty_montage._get_ch_pos()
ch_names = ten_twenty_montage.ch_names

new_names = []
for ch in ch_names:
    ch = ch.lower()
    new_names.append(ch[0].upper() + ch[1:])

ten_twenty_montage.ch_names = new_names

# Remove dots from channel names in raw.info['ch_names']
new_names = []
for ch_name in raw.info['ch_names']:
    new_names.append(ch_name.split('.')[0])

mne.rename_channels(raw.info, dict(zip(raw.ch_names, new_names)))
# Set Montage 10_20 system
raw.set_montage(ten_twenty_montage, verbose=False);

# Sphere not oriented with 10-20 system
# raw.plot_sensors(kind = '3d', sphere=(0.0, 0.015, 0.033, 0.1));

In [32]:
# Parameters to task
pick_ch = bm.C_Channels # Considering only the channels that map to topo map functionality

# Channel names to Indices
ch_names = raw.ch_names
pick_ch_idx = [ch_names.index(i) for i in pick_ch]

# Duplicate params
scale = dict(mag=1e-12, grad=4e-11, eeg=100e-6)
mne_plot_raw = dict(scalings=scale, clipping='transparent', order=pick_ch_idx)

#### Remove Line noise and apply band pass

In [10]:
# Remove and Filter signal noises
elec_lines, L_cutoff, H_cutoff = 60, 1, 50 #HZ
raw.notch_filter(elec_lines)
rawfltrd = raw.filter(L_cutoff, H_cutoff, verbose= False).copy()

# Check the Power spectral density
# rawfltrd.plot_psd();

In [33]:
## Analysis post filter
rawfltrd.pick_channels(pick_ch)
rawfltrd.plot(scalings=scale, clipping='transparent', order=pick_ch_idx);
# rawfltrd.plot(scalings='auto');
# rawfltrd.plot_psd_topo(show = True);
# rawfltrd.plot_sensors(show_names=True);

#### Noise Removal : Signal Space Projection(SSP)

In [8]:
eog_proj, events = mne.preprocessing.compute_proj_eog(rawfltrd, n_grad=0, n_mag=0, n_eeg=2, average=True, verbose=False, ch_name = 'Fpz', reject=None)

In [9]:
rawfltrd.add_proj(projs=eog_proj);
# rawfltrd = rawfltrd.apply_proj();

##### Common Average Reference(CAR) (Projector)

In [10]:
## TODO- Remove bads prior
rawfltrd.set_eeg_reference(ref_channels='average', projection=True);

In [11]:
# check how the projections are
rawfltrd.plot_projs_topomap();
rawfltrd.info['projs']

[<Projection | EOG-eeg--0.200-0.200-PCA-01, active : False, n_channels : 64>,
 <Projection | EOG-eeg--0.200-0.200-PCA-02, active : False, n_channels : 64>,
 <Projection | Average EEG reference, active : False, n_channels : 64>]

In [12]:
# Compare after and before projection
# rawfltrd.plot(scalings=scale, clipping='transparent', order=pick_ch_idx);
rawfltrd.plot(scalings=scale, clipping='transparent', butterfly= False, title='Raw Filtered (Without Projection))', proj= False, order=pick_ch_idx);
rawfltrd.plot(scalings=scale, clipping='transparent', butterfly= False, title='Raw Filtered(With Projection)', proj= True, order=pick_ch_idx);

Channels marked as bad: none
Channels marked as bad: none


#### Noise Removal : Independent Component Analysis(ICA)

In [43]:
rawfltrd.plot_sensors(show_names=True, sphere=(0.0, 0.01, 0.029, 0.1));

In [13]:
# cov = mne.Covariance()
ica = mne.preprocessing.ICA(n_components = 40, noise_cov= None, random_state=2, method='picard',max_iter=500)

In [29]:
# Create an instance of RAW
rawfltrd_ica = rawfltrd.copy()
# rawfltrd_ica.apply_proj()
ica.fit(rawfltrd_ica);
ica.exclude = []

Fitting ICA to data using 64 channels (please be patient, this may take a while)
Selecting by number: 40 components
Fitting ICA took 40.8s.


In [ ]:
# Using EOG Channel to select ICA Components
ica.exclude , ex_scores = ica.find_bads_eog(rawfltrd, ch_name=['Fpz','Fp1','Fp2' ])

In [40]:
## ANalyse ICA components
# ica.plot_components();
# ica.plot_sources(rawfltrd_ica);
# ica.plot_overlay(rawfltrd_ica);
ica.plot_properties(rawfltrd_ica, picks=ica.exclude);

    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
Not setting metadata
61 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Not setting metadata
Not setting metadata
61 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped


In [ ]:
 # ica.exclude = [0,1,2,3] # manually exclude ICA components based on observation on plots above
rawfltrd_ica = ica.apply(rawfltrd_ica)
rawfltrd_ica.plot(scalings=scale, clipping='transparent', title=' ICA', proj= False, order=pick_ch_idx);
# rawfltrd.plot(scalings=scale, clipping='transparent', title='Raw Filtered- w Projection', proj=True, order=pick_ch_idx);
rawfltrd.plot(scalings=scale, clipping='transparent', title='Raw Filtered- wo Projection', proj=False, order=pick_ch_idx);

In [ ]:
## TODO: ICA template matching for Multiple subjects

In [ ]:
rawfltrd_proj = rawfltrd.copy()
rawfltrd_proj.apply_proj()

#### Noise Removal : compare SSP and ICA

In [25]:
# Comparison using data
rawfltrd_df = rawfltrd.to_data_frame(index=['time'])
rawfltrd_proj_df = rawfltrd_proj.to_data_frame(index=['time'])
rawfltrd_ica_df = rawfltrd_ica.to_data_frame(index=['time'])
rawfltrd_ica_df.compare(rawfltrd_proj_df, align_axis=0)

In [91]:
# comparison using plots
dummy = rawfltrd.copy()
data1 = rawfltrd.get_data()
data2 = rawfltrd_ica.get_data()
data3 = rawfltrd_proj.get_data()
dummy._data = data3 - data2
dummy.plot(clipping='transparent', title='DIfference between ICA and SSP', proj=False);

Channels marked as bad: none


### Create Epcohs from events

In [34]:
## TODO: Choose between rawfltrd/ rawfltrd_proj/ rawfltrd_ica
# Capture events from annotations
event_data = mne.events_from_annotations(rawfltrd)
event_marker, event_ids = event_data

Used Annotations descriptions: ['T0', 'T1', 'T2']


In [35]:
# epochs1 = mne.Epochs(rawfltrd, events= event_marker, event_id= event_ids, baseline = (0,0))
epochs = mne.Epochs(rawfltrd, events= event_marker, event_id= event_ids, verbose= False, proj= 'delayed', reject = None) # Baseline is default
# epochs.equalize_event_counts() # Shape = epochs x chan x timepnts

# Evoked data
T0 = epochs['T0'].average() # Shape = chan x timepnts
T1 = epochs['T1'].average()
T2 = epochs['T2'].average()

In [37]:
pick_ch[5]

'Cz'

In [39]:
## Task  Analysis
task = T0
# task.plot_topomap();
# task.plot_white(); # Noise cov required
# task.plot_field(); # requires  surf maps
# task.plot_sensors();
# task.plot_topo();
# task.plot_joint(times=[0.0, 0.2, 0.3]);
T2.plot_image(titles=f'{task} Image');
# task.plot(proj= True, titles = '{task} - Projs - True',spatial_colors=True);
# task.plot(proj= False, titles = '{task} -  Projs - False',spatial_colors=True);
# task.plot(proj= 'reconstruct', titles = '{task} -  Projs - reconstruct',spatial_colors=True);
# task.plot_topomap();
# task.plot(gfp= "only"); # population standard deviation of the signal across channels
## Compare regions
# mne.channels.combine_channels({task}, roi_dict, method='mean')
## Compare conditions
# evoked = dict({task} = list(epochs['{task}'].iter_evoked()), T2 = list(epochs['T2'].iter_evoked()), T0=list(epochs['T0'].iter_evoked()))
# mne.viz.plot_compare_evokeds(evoked, combine='mean')
# task_t0 = mne.combine_evoked([task, T0], weights=[1,-1])
# task_t0.plot_joint();

In [103]:
# epochs.to_data_frame(index=['condition', 'epoch'],long_format=True)
epochs.plot();
epochs.plot_drop_log();

Channels marked as bad: none


In [119]:
# Amplitudes and latency measures
channel, latency, amplitude = T1.get_peak(mode='pos', return_amplitude=True)
print(channel, latency, amplitude)

F5 0.41875 4.826372575954073e-05


In [ ]:
## Data whitening
noise_cov = mne.compute_covariance(epochs, tmax=0., method='shrunk', rank=None,
                                   verbose='error')
T1.plot_white(noise_cov=noise_cov); # step of scaling the whitened plots to show how well the assumption of Gaussian noise is satisfied by the data
 

### Feature Extraction:
Filter Bank CSP

Reimannian Geometry